In [33]:
import joblib
import pandas as pd
import numpy as np
from tf2_data_vis.team_division.steamid import convert_steamid
from tf2_data_vis.get_log_data.batch import join_rgl_info_batch
import os


players = pd.read_csv("../data/players.csv")

player_rounds = pd.read_csv("../data/player_rounds.csv")

rounds = pd.read_csv("../data/rounds.csv")


valid_ids = pd.read_csv("../data/valid_ids.csv")

push_stats = pd.read_csv("../data/push_stats.csv")

teams = pd.read_csv("../data/teams.csv")

healspread = pd.read_csv("../data/healspread.csv")

info = pd.read_csv("../data/info.csv")

round_events = pd.read_csv('../data/round_events.csv')

koth_rounds = pd.read_csv('../data/koth_rounds.csv')


model_ready_data = joblib.load("../data/pkls/model_ready_data_dict.pkl")
X = model_ready_data['X']
players_wide  = model_ready_data['players_wide']


log_ids = joblib.load("../data/pkls/log_id/log_id.pkl")

In [34]:
X

,demoman_dapd_map_adj,scout_pocket_dapd_map_adj,scout_roamer_dapd_map_adj,soldier_pocket_dapd_map_adj,soldier_roamer_dapd_map_adj,demoman_dapm_map_adj,scout_pocket_dapm_map_adj,scout_roamer_dapm_map_adj,soldier_pocket_dapm_map_adj,soldier_roamer_dapm_map_adj,...,medic_pyro_assists_class_kdapd_map_adj,medic_avg_time_before_healing_map_adj,medic_avg_time_to_build_map_adj,medic_avg_uber_length_map_adj,medic_biggest_advantage_lost_map_adj,medic_deaths_with_95_99_uber_rate_map_adj,medic_deaths_within_20s_after_uber_rate_map_adj,medic_drops_rate_map_adj,medic_uberspm_map_adj,weights
0,0.533466,0.560405,0.420340,0.754930,0.719723,0.477755,0.418087,0.417310,0.322108,0.355263,...,0.484857,0.693631,0.425502,0.501799,0.503442,0.514577,0.594906,0.543199,0.538016,2.000000
1,0.166017,0.347390,0.237621,0.546378,0.391001,0.351762,0.537892,0.418224,0.687067,0.547402,...,0.489387,0.578130,0.425828,0.444304,0.502820,0.518098,0.607834,0.464287,0.123561,1.500000
2,0.718185,0.922222,0.598864,0.757380,0.568682,0.594301,0.669519,0.545702,0.687067,0.547402,...,0.489387,0.433151,0.503378,0.444304,0.508022,0.518098,0.607834,0.484487,0.549218,2.250000
3,0.312943,0.560163,0.236159,0.537719,0.561824,0.358015,0.533489,0.348554,0.557670,0.437973,...,0.486423,0.482182,0.521529,0.542429,0.508886,0.512003,0.605029,0.547269,0.742622,1.750000
4,0.529854,0.329656,0.419153,0.350769,0.561824,0.471755,0.428070,0.424651,0.468950,0.537561,...,0.486423,0.427842,0.521529,0.500292,0.508886,0.512003,0.605029,0.479954,0.546721,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
32646,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
32647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.333333
32648,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000


In [2]:
# from tf2_data_vis.player_division.request_rgl import request_loop
# import os
# rgl_info = request_loop(sleep = .5,parent_dir=os.getcwd(),output_dir='data')

In [3]:
rgl = join_rgl_info_batch(parent_dir=os.getcwd(),output_dir='data',batch_type='rgl_info')

In [4]:
rgl_df = pd.concat([pd.Series(rgl['id']),pd.Series(rgl['info'])],axis = 1)
rgl_df.columns = ['steamid','info']
no_profile = rgl_df[rgl_df['info'] == 'No Profile']
profile = rgl_df.drop(no_profile.index)

# Grab the first JSON in each tuple
profile['rgl_profile'] = profile['info'].apply(lambda x: x[0].json())
profile['teams'] = profile['info'].apply(lambda x: x[1].json())

In [5]:
team_dfs = profile['teams'].apply(pd.json_normalize)

team_dfs = pd.concat([profile['steamid'],team_dfs],axis = 1)

In [6]:
def add_steamid(inner_df, steamid):
    inner_df = inner_df.copy()  # avoid modifying original in-place
    inner_df['steamid'] = steamid
    return inner_df

# Apply to the Series
team_dfs['teams_id'] = team_dfs.apply(lambda row: add_steamid(row['teams'], row['steamid']), axis=1)

team_df_list = team_dfs['teams_id'].to_list()

rgl_teams = pd.concat(team_df_list)
rgl_teams = rgl_teams[['steamid'] + [col for col in rgl_teams.columns if col != 'steamid']]
sixes_teams = rgl_teams[rgl_teams['formatName'] == "Sixes"].copy()
useful_cols = ['steamid', 'formatId', 'formatName', 'regionId', 'regionName','startedAt','leftAt','divisionId', 'divisionName',
       'teamId']
sixes_teams = sixes_teams[useful_cols]


In [7]:
import re

# Make a copy to avoid SettingWithCopyWarning
sixes_teams = sixes_teams.copy()
sixes_teams['divisionName'] = sixes_teams['divisionName'].str.lower()

# Replacement mapping
from_list = ['im/am', 'am/nc', 'adv', 'division one']
to_list = ['amateur', 'newcomer', 'advanced', 'invite']

# Replace using word boundaries
for f, t in zip(from_list, to_list):
    pattern = r'\b' + re.escape(f) + r'\b'  # ensures exact match
    sixes_teams['divisionName'] = sixes_teams['divisionName'].str.replace(pattern, t, regex=True)


In [8]:
divisions = ['amateur','main','intermediate','advanced','newcomer','invite']

sixes_teams = sixes_teams[sixes_teams['divisionName'].isin(divisions)]


In [10]:
players['steamid64'] = players['steamid'].apply(convert_steamid)
sixes_teams['steamid64'] = sixes_teams['steamid'].copy()
sixes_teams['startedAt'] = pd.to_datetime(sixes_teams['startedAt']).dt.tz_localize(None)
sixes_teams['leftAt'] = pd.to_datetime(sixes_teams['leftAt']).dt.tz_localize(None)
sixes_teams['leftAt'] = sixes_teams['leftAt'].fillna(pd.Timestamp.now())
player_teams = players.merge(sixes_teams,on = 'steamid64',how = 'inner')
player_teams = player_teams.merge(info[['id','date']],on = 'id')
player_teams['date'] = pd.to_datetime(player_teams['date'])


In [11]:
player_teams['near_start'] = (player_teams['date'] - player_teams['startedAt']).dt.total_seconds().abs() / (60 * 60 * 24)
player_teams['near_left'] = (player_teams['date'] - player_teams['leftAt']).dt.total_seconds().abs() / (60 * 60 * 24)

In [12]:
closest_team = player_teams.groupby(['id','steamid64']).agg(
    near_start_idx=('near_start', 'idxmin'),
    near_left_idx=('near_left', 'idxmin'),
    near_start_min=('near_start', 'min'),
    near_left_min=('near_left', 'min')
).reset_index()


In [13]:
divisions =  player_teams.loc[closest_team['near_start_idx'],'divisionName'].copy()

In [14]:
closest_team['division'] = divisions.values

In [16]:
player_divisions = players.merge(closest_team[['id','steamid64','division']],on = ['id','steamid64'],how = 'left')

In [18]:
player_divisions['division'] = player_divisions['division'].fillna('NA')

In [21]:
division_weights ={
    "NA":np.nan,
    "newcomer":1,
    'amateur':1.5,
    'intermediate':2,
    'main':3,
    'advanced':5,
    'invite': 10
}

player_divisions['division_weight'] = player_divisions['division'].map(division_weights)


In [22]:
team_divisions = player_divisions.groupby(['id','team'])['division_weight'].mean()

In [ ]:
team_divisions = team_divisions.fillna(1).reset_index()